<br>

# <center> YOLOX + YOLOv5 Weighted Boxes Fusion Ensemble</center>

<div align = 'center'><img src='https://i.ibb.co/tZWZ3v0/ppap.png'/></div>

<br>
<hr>
<br>



-----

**Important Note:** Please note that the higest LB score is coming from an older version where a model in the ensemble had priority over the other (resulting in one model taking over most of the predictions). 
The current version has the correct score (that is 0.447). Apologies for the inconvenience. 

-----

### How to ensemble object detection models?

This notebook shows how to detect starfish objects (COTS dataset) using multiple models on Kaggle (YOLOX & YOLOv5 for example). 

<hr>

**Original Notebooks Credits:**
I have used the great models from [YoloX full training pipeline for COTS dataset](https://www.kaggle.com/remekkinas/yolox-full-training-pipeline-for-cots-dataset) and from [YoloX training pipeline COTS dataset [LB 0.507] !!
](https://www.kaggle.com/remekkinas/yolox-training-pipeline-cots-dataset-lb-0-507).
And the ensemble is from [WBF approach for ensemble](https://www.kaggle.com/shonenkov/wbf-approach-for-ensemble/notebook).
<hr>


Following [the original notebook](https://www.kaggle.com/shonenkov/wbf-approach-for-ensemble/notebook), here is a super easy technique for bbox postprocessing ensembling object detection tasks.

Original paper: [Weighted Boxes Fusion: ensembling boxes for object detection models](https://arxiv.org/pdf/1910.13302.pdf)


**MAIN IDEA**
We know about NMS (Non-maximum Suppression) method and its [Soft-NMS](https://arxiv.org/pdf/1704.04503.pdf) extension. But on this notebook we will use WBF (Weighted Boxes Fusion), that gives really good boost in [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html) according to [paper](https://www.kaggleusercontent.com/kf/33938688/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..DAfPciYx8Oc7kJrKcYawig.uvFe9prASYZNcVo76gM8ut-7s5bGRQ-lUXSOWGU5EflkWsk-TpJKhYUObxe6ntB0_--EImt8-pehmt1tQEbdNeQPePCwwWz9fQhny16RsdDQvlz4dGFXVLMsW4lnbVjdZyhfmeXp-wLpf7yZPJ-9xQ9GnUDvCiy21i0O_V-RVFRlEUCvw9NDCBFawHNaOEGiRcltaPrN1fdlnfrj7OW-v8qYdMLE-rUjvl3Q19KHCkuh81EV7GgkHdHbeTXMG45hNAuuEo2WoFSvT7djDwxUvQUxPsGAPpZc7-hXrmw4IJFwtrhwqdatecFw6HD67Q8HObWPyCWtd5tihPK2gkMa7r7YSU4Nejgl293muJCMMlN6XnWfRKweK5T6z6df1-dTfSWxtUTHtGWV1RfGxT8NN-KV_7cpeGY2kMGRPdk4lbqreQCeGLjtbmUeBiNhgTX1acodY6JfN2kGSFPn15S22GqqQFOdKEaLnaNZGZ0c3IAHgyuJE4yfsAjC3wrFAPQz25y1u6E4k1e8yQlVklIhCdBKCl8zkvDffgXAzl-POm_N_AeW6zVPhVZ_86ipENK7eqcjD2VU5oxi2S7IclRdn_eEulBPrrhlpjl0E8axDoEkGE_R_uncF3uZQdAG6AYg2PwgoEGGxTo_uTk5ul3Op3CfdPwPC9NTvLZ4qxR9T8E.a8_2yDGoW9UBcfsWU7txuA/(https://arxiv.org/pdf/1910.13302.pdf).

#### Weighted boxes fusion (WBF) Algorithm

1. Each predicted box from each model is added to a single list B. The list is sorted in decreasing order of the confidence scores C.

2. Create lists: L and F for boxes clusters and fused boxes. Each position in L contains a set of boxes (or single box), which form a cluster.  Each position in F contains only one box, which is the fused box from the corresponding cluster in L. 

3.  Iterate predicted boxes in B in cycle and try to find a matching box in the list F.  Match is defined as a box with a large overlap with the box under question (IoU > thresh).

4. If the match is not found, add the box from the list B to the end of lists L and F as new entries; proceed to the next box in the list B.

5. If the match is found, add this box to the list L at the position pos corresponding to the matching box in the list F.

6. Recalculate the box coordinates and confidence score in F[pos], using all T boxes accumulated in cluster L[pos]. (see the formula in the paper)

7. After all boxes in B are processed, re-scale confidence scores in F list: multiply it by a number of boxes in a cluster and divide by a number of models N. (see the formula in the paper)


**When WBF can be better than NMS or SoftNMS?**
Both NMS and Soft-NMS exclude some boxes, but WBF uses information from all boxes. It can fix some cases where all boxes are predicted inaccurate by all models. NMS will leave only one inaccurate box, while WBF will fix it using information from all 3 boxes (see the example in Fig. 1, red predictions, blue ground truth).


![](https://i.ibb.co/d2P2pPL/2020-05-12-21-02-34.png)

In [1]:
import warnings; warnings.filterwarnings("ignore")

import os
import cv2
import ast
import sys
import glob
import torch
import shutil
import importlib
import traceback
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
sys.path.append('../input/tensorflow-great-barrier-reef')
tqdm.pandas()

from PIL import Image
from IPython.display import display

### Installing YOLOX 
<div class="alert alert-warning" role="alert"><strong>Some Kaggle enviroment hacking :) due to competition limitation - no internet access during submission.</strong></div>

In [2]:
%cp -r /kaggle/input/yolox-cots-models /kaggle/working/
%cd /kaggle/working/yolox-cots-models/yolox-dep

/kaggle/working/yolox-cots-models/yolox-dep


In [3]:
# Install YOLOX required modules
!pip install pip-21.3.1-py3-none-any.whl -f ./ --no-index
!pip install loguru-0.5.3-py3-none-any.whl -f ./ --no-index
!pip install ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl -f ./ --no-index
!pip install onnx-1.8.1-cp37-cp37m-manylinux2010_x86_64.whl -f ./ --no-index
!pip install onnxruntime-1.8.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl -f ./ --no-index
!pip install onnxoptimizer-0.2.6-cp37-cp37m-manylinux2014_x86_64.whl -f ./ --no-index
!pip install thop-0.0.31.post2005241907-py3-none-any.whl -f ./ --no-index
!pip install tabulate-0.8.9-py3-none-any.whl -f ./ --no-index

Looking in links: ./
Processing ./pip-21.3.1-py3-none-any.whl
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
Looking in links: ./
Processing ./loguru-0.5.3-py3-none-any.whl
Looking in links: ./
Processing ./ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl
Looking in links: ./
Processing ./onnx-1.8.1-cp37-cp37m-manylinux2010_x86_64.whl
Looking in links: ./
Processing ./onnxruntime-1.8.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: ./
Processing ./onnxoptimizer-0.2.6-cp37-cp37m-manylinux2014_x86_64.whl
Looking in links: ./
Processing ./thop-0.0.31.post2005241907-py3-none-any.whl
Looking in links: ./
Processing ./tabulate-0.8.9-py3-none-any.whl
tabulate is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [4]:
# Install YOLOX
%cd /kaggle/working/yolox-cots-models/YOLOX
!pip install -r requirements.txt
!pip install -v -e . 

/kaggle/working/yolox-cots-models/YOLOX
Using pip 21.3.1 from /opt/conda/lib/python3.7/site-packages/pip (python 3.7)
Obtaining file:///kaggle/working/yolox-cots-models/YOLOX
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-oc8rsxoe/yolox.egg-info
  writing /tmp/pip-pip-egg-info-oc8rsxoe/yolox.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-oc8rsxoe/yolox.egg-info/dependency_links.txt
  writing top-level names to /tmp/pip-pip-egg-info-oc8rsxoe/yolox.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-oc8rsxoe/yolox.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-oc8rsxoe/yolox.egg-info/SOURCES.txt'
  adding license file 'LICENSE'
  writing manifest file '/tmp/pip-pip-egg-info-oc8rsxoe/yolox.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Running setup.py develop for yolox
    Running command /opt/conda/bin/python3.7 -c 'import io, os, sys, setuptools, tokenize; sy

In [5]:
# Install CocoAPI tool
%cd /kaggle/working/yolox-cots-models/yolox-dep/cocoapi/PythonAPI
!make
!make install
!python setup.py install

/kaggle/working/yolox-cots-models/yolox-dep/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/opt/conda/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /kaggle/working/yolox-cots-models/yolox-dep/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
gcc -pthread -B /opt/conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/lib/python3.7/site-packages/numpy/core/include -I../common -I/opt/conda/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -

In [6]:
import pycocotools

### Test Model

In [7]:
%cd /kaggle/working/yolox-cots-models/YOLOX

CHECKPOINT_FILE = '/kaggle/working/yolox-cots-models/yx_l_003.pth'

/kaggle/working/yolox-cots-models/YOLOX


In [8]:
config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 1
        self.width = 1
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        self.num_classes = 1

'''

with open('cots_config.py', 'w') as f:
    f.write(config_file_template)

In [9]:
from yolox.utils import postprocess
from yolox.data.data_augment import ValTransform

COCO_CLASSES = (
  "starfish",
)

# get YOLOX experiment
current_exp = importlib.import_module('cots_config')
exp = current_exp.Exp()

# set inference parameters
test_size = (800, 1280)
num_classes = 1
confthre = 0.25
nmsthre = 0.4


# get YOLOX model
yolox_model = exp.get_model()
yolox_model.cuda()
yolox_model.eval()

# get custom trained checkpoint
ckpt_file = CHECKPOINT_FILE
ckpt = torch.load(ckpt_file, map_location="cpu")
yolox_model.load_state_dict(ckpt["model"])

<All keys matched successfully>

In [10]:
def yolox_inference(img, model, test_size): 
    bboxes = []
    bbclasses = []
    scores = []
    
    preproc = ValTransform(legacy = False)

    tensor_img, _ = preproc(img, None, test_size)
    tensor_img = torch.from_numpy(tensor_img).unsqueeze(0)
    tensor_img = tensor_img.float()
    tensor_img = tensor_img.cuda()

    with torch.no_grad():
        outputs = model(tensor_img)
        outputs = postprocess(
                    outputs, num_classes, confthre,
                    nmsthre, class_agnostic=True
                )

    if outputs[0] is None:
        return [], [], []
    
    outputs = outputs[0].cpu()
    bboxes = outputs[:, 0:4]

    bboxes /= min(test_size[0] / img.shape[0], test_size[1] / img.shape[1])
    bbclasses = outputs[:, 6]
    scores = outputs[:, 4] * outputs[:, 5]
    
    return bboxes, bbclasses, scores

In [11]:
def draw_yolox_predictions(img, bboxes, scores, bbclasses, confthre, classes_dict):
    for i in range(len(bboxes)):
            box = bboxes[i]
            cls_id = int(bbclasses[i])
            score = scores[i]
            if score < confthre:
                continue
            x0 = int(box[0])
            y0 = int(box[1])
            x1 = int(box[2])
            y1 = int(box[3])

            cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)
            cv2.putText(img, '{}:{:.1f}%'.format(classes_dict[cls_id], score * 100), (x0, y0 - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, (0,255,0), thickness = 1)
    return img

# YOLOv5 Model

In [12]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
CKPT_PATH = '/kaggle/input/10000-resolution/best.pt'
IMG_SIZE  = 1280
CONF      = 0.25
IOU       = 0.50
AUGMENT   = False

In [13]:
def get_path(row):
    row['image_path'] = f'{ROOT_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

In [14]:
# Train Data
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df = df.progress_apply(get_path, axis=1)
df['annotations'] = df['annotations'].progress_apply(lambda x: ast.literal_eval(x))

  0%|          | 0/23501 [00:00<?, ?it/s]

  0%|          | 0/23501 [00:00<?, ?it/s]

In [15]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts()/len(df)*100

  0%|          | 0/23501 [00:00<?, ?it/s]

In [16]:
# norfair dependencies
%cd /kaggle/input/norf1111/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norf1111/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norf1111/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index
%cd ..

print('done')

/kaggle/input/norf1111
Looking in links: ./
Processing ./commonmark-0.9.1-py2.py3-none-any.whl
Processing ./rich-9.13.0-py3-none-any.whl
/kaggle/working/tmp/filterpy-1.4.5
Processing /kaggle/working/tmp/filterpy-1.4.5
  Preparing metadata (setup.py) ... - done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=b0dedda44c24a14f334b3470d5ed469e37eb23b40dea34028ff1a5da70fccf9b
  Stored in directory: /root/.cache/pip/wheels/0e/ba/6d/20557ee403bc0c849d9fe3b358ded9918d7d2b0c5b0253fc99
Successfully built filterpy
/kaggle/input/norf1111
Looking in links: ./
Processing ./norfair-0.3.1-py3-none-any.whl
/kaggle/input
done


In [17]:
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
sys.path.append('../')
from norfair import Detection, Tracker
import torch

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))

    return result


# Euclidean distance function to match detections on this frame with tracked_objects from previous frames
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)


def tracking_function(tracker, frame_id, bboxes, scores):
    detects = []
    predictions = []

    if len(scores) > 0:
        for i in range(len(bboxes)):
            box = bboxes[i]
            score = scores[i]
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            detects.append([x_min, y_min, x_min + bbox_width, y_min + bbox_height, score])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    #             print(predictions[:-1])
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
        score = tobj.last_detection.scores[0]

        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))

    return predictions


tracker = Tracker(
    distance_function=euclidean_distance,
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)

In [18]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    # converstion (xmid, ymid) => (xmin, ymin)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes

def coco2voc(bboxes, image_height=720, image_width=1280):
    bboxes  = coco2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2voc(bboxes, image_height, image_width)
    return bboxes

def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2):

    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors

    if bbox_format == 'yolo':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:

                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)

    elif bbox_format == 'coco':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)

    elif bbox_format == 'voc_pascal':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [19]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [20]:
def load_model(ckpt_path, conf=0.25, iou=0.50):
    model = torch.hub.load('/kaggle/input/yolov5-lib-ds',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
#     model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
#     model.multi_label = False  # NMS multiple labels per box
#     model.max_det = 1000  # maximum number of detections per image
    return model

In [21]:
def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]

def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def show_img(img, bboxes, bbox_format='yolo', bbox_colors = None):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes,
                           classes = names,
                           class_ids = labels,
                           class_name = True,
                           colors = colors if bbox_colors is None else bbox_colors,
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

# Ensembling

## Let's look at some train images!

In [22]:
import sys; sys.path.append('/kaggle/input/weightedboxesfusion/')
# !pip install --no-deps '/kaggle/input/weightedboxesfusion/' > /dev/null

In [23]:
from ensemble_boxes import *
import numpy

# [[     235.27      236.26      38.689      36.261]] tensor([0.50011]) [array([236, 235,  38,  35])] [0.5798921585083008]
# boxes_list = [[[235.27,236.26,38.689,36.261]], [[236, 235,  38,  35]]]
# boxes_list = [[[235,236,38,36]], [[236, 235,  38,  35]]]
# boxes_list = [[[235,236,38,36]], [[235,236,38,36]]]
# boxes_list = [[numpy.array([0.235,0.236,0.38,0.36])], [numpy.array([0.237,0.236,0.38,0.36])]]  
# 0.18395,     0.18472,    0.030249,    0.028351]]), array([[    0.18452,     0.18374,    0.029711,    0.027365
# boxes_list = [[numpy.array([0.18395,0.18472,0.030249,0.028351])], [numpy.array([0.18452,0.18374,0.029711,0.027365])]]  
# scores_list = [[0.50011], [0.5798921585083008]]

boxes_list = [[numpy.array([0.18,0.18,0.03,0.92])], [numpy.array([0.18,0.18,0.03,0.92])]]  
scores_list = [[0.50], [0.6]]
labels_list = [[1],[1]]

# boxes_list = [[
#     [0.00, 0.51, 0.81, 0.91],
#     [0.10, 0.31, 0.71, 0.61],
#     [0.01, 0.32, 0.83, 0.93],
#     [0.02, 0.53, 0.11, 0.94],
#     [0.03, 0.24, 0.12, 0.35],
# ],[
#     [0.04, 0.56, 0.84, 0.92],
#     [0.12, 0.33, 0.72, 0.64],
#     [0.38, 0.66, 0.79, 0.95],
#     [0.08, 0.49, 0.21, 0.89],
# ]]

# scores_list = [[0.9, 0.8, 0.2, 0.4, 0.7], [0.5, 0.8, 0.7, 0.3]]
# labels_list = [[0, 1, 0, 1, 1], [1, 1, 1, 0]]
weights = [2, 1]

iou_thr = 0.1
skip_box_thr = 0.0001
sigma = 0.1

boxes, scores, labels = nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)
print(boxes, scores, labels)
boxes, scores, labels = soft_nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
print(boxes, scores, labels)
boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
print(boxes, scores, labels)
boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
print(boxes, scores, labels)

[[0.18 0.18 0.03 0.92]
 [0.18 0.18 0.03 0.92]] [0.33333333 0.2       ] [1 1]
[[0.18 0.18 0.03 0.92]
 [0.18 0.18 0.03 0.92]] [0.22222222 0.06666667] [1 1]
[[nan nan nan nan]
 [nan nan nan nan]] [0.2962963  0.02222222] [1. 1.]
[[0.18 0.18 0.03 0.92]
 [0.18 0.18 0.03 0.92]] [0.09876543 0.00740741] [1. 1.]


In [24]:
TEST_IMAGE_PATH = "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/9674.jpg"

from ensemble_boxes import *
import numpy
import os 

def run_wbf(bboxes, confs, image_size=1280, iou_thr=0.2, skip_box_thr=0.001, weights=None):
    boxes =  [numpy.array(bbox)/(image_size) for bbox in bboxes]
    
    scores = [numpy.array(conf) for conf in confs]
    labels = [np.ones(numpy.array(conf).shape[0]) for conf in confs]
    
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=[1,1.5], iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    boxes = boxes*(image_size-1)
    boxes=voc2coco(boxes).astype(int)
    #print("voc:coco",boxes)
    return boxes, scores, labels

model = load_model(CKPT_PATH, conf=CONF, iou=IOU)


def test_folder():
    # 批量run
    #1111111111111
    image_paths = "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/"
    frame_id = 0
    for idx, path in enumerate(os.listdir(image_paths)):
    #     print(path)
        image_np = cv2.imread(image_paths+path)[...,::-1]
    #     try: 
        bboxes_1, bbclasses, scores = yolox_inference(image_np[...,::-1], yolox_model, (800,1280))

        bboxes_2, confis = predict(model, image_np, size=9000, augment=AUGMENT)
        new_bboxes_2, new_confs = [], []
        for i, conf in enumerate(confis):
            if conf > 0.28:
                new_bboxes_2.append(bboxes_2[i])
                new_confs.append(conf)


        if len(bboxes_1) > 0 and len(new_bboxes_2) > 0: 
            # run_wbf 只接受voc格式： x1 y1 x2 y2
            # yolox 本身输出voc格式， yolov5 经过转换输出了 coco 需要转为voc
            pred_1, pred_2 = bboxes_1.detach().numpy(), coco2voc(numpy.array(new_bboxes_2))

            print('yolox')
            print(pred_1, scores)
            display(show_img(image_np, pred_1, bbox_format='voc_pascal'))  

            print('yolov5')
            print(pred_2, new_confs)
            display(show_img(image_np, pred_2, bbox_format='voc_pascal')) 

            # run_wbf输出已经在函数里转回了coco
            bboxes, scores, labels = run_wbf([pred_1, pred_2], [scores.numpy(), new_confs], image_size = IMG_SIZE)   # 中午到这里！！加了个[0]

            print('final')
            print(bboxes, scores)
            display(show_img(image_np, bboxes, bbox_format='coco')) 



        elif len(bboxes_1) > 0: bboxes, scores = voc2coco(bboxes_1.detach().numpy(), image_np.shape[1], image_np.shape[2]).astype(int), scores        
        elif len(new_bboxes_2) > 0: bboxes, scores = new_bboxes_2, new_confs        
        else: bboxes, scores = [], []
        # display(show_img(image_np, bboxes, bbox_format='coco'))   

    #     print('final = ', bboxes, scores)

        predictions = tracking_function(tracker, frame_id, bboxes, scores)
        prediction_str = ' '.join(predictions)
        print('after tracking = ', prediction_str)
        print('-------------------------')
        
        frame_id += 1
    #111111111111111

# test_folder()


YOLOv5 🚀 2022-1-18 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients, 16.2 GFLOPs
Adding AutoShape... 


## SUBMIT PREDICTION TO COMPETITION

In [25]:
%cd /kaggle/working/
!pwd

/kaggle/working
/kaggle/working


In [26]:
import greatbarrierreef

env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [27]:
os.chdir('../working/')
!pwd

/kaggle/working


In [28]:
submission_dict = {
    'id': [],
    'prediction_string': [],
}  
    
frame_id = 0
for (image_np, sample_prediction_df) in iter_test:
    
    bboxes_1, bbclasses, scores = yolox_inference(image_np[...,::-1], yolox_model, test_size)
    bboxes_2, confis = predict(model, image_np, size=9000, augment=AUGMENT)
    new_bboxes_2, new_confs = [], []
    for i, conf in enumerate(confis):
        if conf > 0.28:
            new_bboxes_2.append(bboxes_2[i])
            new_confs.append(conf)
    
    if len(bboxes_1) > 0 and len(new_bboxes_2) > 0: 
        pred_1, pred_2 = bboxes_1.detach().numpy(), coco2voc(numpy.array(new_bboxes_2))
        bboxes, scores, labels = run_wbf([pred_1, pred_2], [scores.numpy(), new_confs], image_size = IMG_SIZE)   # 中午到这里！！加了个[0]
    elif len(bboxes_1) > 0: bboxes, scores = voc2coco(bboxes_1.detach().numpy(), image_np.shape[1], image_np.shape[2]).astype(int), scores        
    elif len(new_bboxes_2) > 0: bboxes, scores = new_bboxes_2, new_confs        
    else: bboxes, scores = [], []
    
    predictions = tracking_function(tracker, frame_id, bboxes, scores)
    prediction_str = ' '.join(predictions)
    
    
    print('prediction_str')
    sample_prediction_df['annotations'] = prediction_str

    env.predict(sample_prediction_df)
    frame_id += 1


#         predictions = []
#         for i in range(len(bboxes)):
#             box = bboxes[i]        
#             score = scores[i]
            
#             if score > 0.1:
            
#                 x_min = int(box[0])
#                 y_min = int(box[1])
#                 bbox_width = int(box[2])
#                 bbox_height = int(box[3])

#                 predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))

#         prediction_str = ' '.join(predictions)
#     except:
#         traceback.print_exc()
#         predictions = []
#     sample_prediction_df['annotations'] = prediction_str
#     env.predict(sample_prediction_df)

#     print('Prediction:', prediction_str)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
prediction_str
prediction_str
prediction_str


In [29]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()

,index,annotations
0,0,NaN
1,1,NaN
2,2,NaN


<div class="alert alert-success" role="alert">
    Find this notebook helpful? :) Upvoting is FREE ;) 
 </div>